- 본 과정에서는 "Robust regression"을 다른 regression과 비교하여 볼 것입니다.

regression은 임의에 x 값이 변화됩에 따라 y값의 변화량이 선형적($y=ax^3+bx^2+cx+\epsilon\ \ or\ \ y=ax^2+bx+\epsilon\ \ or\ \ y=ax+\epsilon.\ \ $ etc)인지를 확률(p-value)로 정의한다. 따라서, 오차가 존재하고 더불어, 이상치에 따라 예측이 외곡될 수도 있다.

Robust regression은 이상치 탐색과 함께 회귀가 가능하다.
회귀의 기본적인 원리는 각 변수 확률적 분포에 대한 비교라고 볼 수도 있습니다. 이런 이유로 이상치(outlier)는 '평균', '편차'와 같은 변수에 대해 'summary statistics'를 왜곡할 수 있고, 분포의 기준이 되는 '중심값'을 외곡시키기 때문에 해당 변수의 분포의 변화를 초래하여  점에서 고질적인 문제가 됩니다. 

해당 문제를 해결하기 위한 방법으로 이상치 예상을 구체적으로 해결하는 수정된 선형회귀(Robust regression)을 사용합니다. 

In [3]:
from random import random
from random import randint
from random import seed
from sklearn.datasets import make_regression
from matplotlib import pyplot

In [4]:
X, y = make_regression(n_samples=100, n_features=1, tail_strength=0.9, effective_rank=1, n_informative=1, noise=3, bias=50, random_state=1)

In [35]:
# 데이터에 임의의 이상치를 추가합니다.
seed(1)
for i in range(10):
    # range(2, 5) {2 <= x < 5} 범위에 대한 임의의 수를 생성합니다.     
    factor = randint(2, 4)
    # 의의로 생성한 난수에 X의 표준편차를 곱한 수를 i번째 독립변수에 더하거나 빼줍니다.     
    if random() > 0.5:
        X[i] += factor * X.std()
    else:
        X[i] -= factor * X.std()        